In [ ]:
%load_ext tensorboard

In [29]:
import numpy as np
import pandas as pd 
from ast import literal_eval
import tensorflow as tf
from tensorflow.keras.preprocessing import text, sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding, Dense, LSTM, MaxPooling1D, Input, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.layers import Bidirectional, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.metrics import AUC
import datetime, os

In [30]:
MAX_VOCAB_SIZE = 200000 # there are 563693 words in the vocabulary
MAX_LEN_SEQ = 300
TRAIN_TEXT_COL = 'comment_text_clean2'
TEST_TEXT_COL = 'comment_text_clean2'
TRAIN_TARGET_COL = 'target'
EMBED_DIM = 300
EMBEDDING_FILE = 'embeds/glove.840B.300d.txt'

DROPOUT_RATE = 0.2
LSTM_UNITS = 128
BATCH_SIZE = 128
NUM_EPOCHS = 4
CHECKPOINT_PATH = "NN_models/cp.ckpt"
CHECKPOINT_DIR = os.path.dirname(CHECKPOINT_PATH)


In [31]:
# Load dataframe
train_data = pd.read_csv('data/train_for_nn.csv', converters={"comment_text_clean2": literal_eval})
test_data = pd.read_csv('data/test_for_nn.csv', converters={"comment_text_clean2": literal_eval})

In [33]:
# Create train val split, stratify on target
train_df, val_df = train_test_split(train_data, test_size=0.2, stratify=train_data['target'], random_state=1)

In [34]:
# Create and fix tokenizer
def train_tokenizer(train_data, vocab_size):
    # Use Keras tokenizer to create vocabulary dictionary 
    # default arguments will filter punctuation and convert to lower, we do not want this given our use 
    # of pre-trained word embeddings
    tokenizer = text.Tokenizer(num_words = vocab_size, filters='', lower=False)
    tokenizer.fit_on_texts(train_data)
    return tokenizer

# pad tokenized sequences
def text_padder(text, tokenizer):
    return sequence.pad_sequences(tokenizer.texts_to_sequences(text), maxlen=MAX_LEN_SEQ)

# Build embedding matrix
def build_embedding_matrix(word_indexes, EMBEDDING_FILE):
  
    # Used to store words as key and vectors as value
    embedding_dict = {}
    with open(EMBEDDING_FILE) as file:
        # file is formatted word {whitespace} vector
        for line in file:
            pairs = line.split(' ')
           # word is 0 index of pairs
            word = pairs[0]
            vec = pairs[1:]
           #convert vec into a numpy array
            vec = np.asarray(vec, dtype=np.float32)
            embedding_dict[word] = vec
    
    #create the embedding matrix which has dimensions:
    # MAX_VOCAB_SIZE +1 for rows, this means there will be as many rows as words we allow to be part of the feature set.
    # EMBED_DIM is the number of columns, this reflects the dimensions of the word embedding vectors we are using.
    embedding_matrix = np.zeros((len(word_indexes)+1, EMBED_DIM))


    word_count = 0
    for word, i in word_indexes.items():
        # gets the vector to the corresponding word from the previous dictionary and sets it to the variable
        embedding_vector = embedding_dict.get(word)
        # We check whether the embedding_vector is not none (i.e the word is in the embedding index)
        if embedding_vector is not None:
            word_count += 1
            # Append the embedding vector to index i in the embedding matrix 
            embedding_matrix[i] = embedding_vector
            
            
def build_embedding_matrix_restricted(word_indexes, EMBEDDING_FILE):
  
    # Used to store words as key and vectors as value
    embedding_dict = {}
    with open(EMBEDDING_FILE) as file:
        # file is formatted word {whitespace} vector
        for line in file:
            pairs = line.split(' ')
           # word is 0 index of pairs
            word = pairs[0]
            vec = pairs[1:]
           #convert vec into a numpy array
            vec = np.asarray(vec, dtype=np.float32)
            embedding_dict[word] = vec
    
    #create the embedding matrix which has dimensions:
    # MAX_VOCAB_SIZE +1 for rows, this means there will be as many rows as words we allow to be part of the feature set.
    # EMBED_DIM is the number of columns, this reflects the dimensions of the word embedding vectors we are using.
    embedding_matrix = np.zeros((MAX_VOCAB_SIZE+1, EMBED_DIM))


    word_count = 0
    for word, i in word_indexes.items():
        # gets the vector to the corresponding word from the previous dictionary and sets it to the variable
        embedding_vector = embedding_dict.get(word)
        # We check whether the embedding_vector is not none (i.e the word is in the embedding index)
        if embedding_vector is not None:
            word_count += 1
            # Append the embedding vector to index i in the embedding matrix 
            embedding_matrix[i] = embedding_vector

In [36]:
tokenizer = train_tokenizer(train_df[TRAIN_TEXT_COL], MAX_VOCAB_SIZE)

In [38]:
len(tokenizer.word_index)

494877

In [107]:
# build model

# NOTE: WITH TF2.0 CUDNNLSTM is active by default when there is a GPU available but you must use the default settings.
# SEE https://www.tensorflow.org/api_docs/python/tf/keras/layers/LSTM for more details

def build_model(embedding_matrix):
    # change to max word length 
    input_words = Input(shape=(MAX_LEN_SEQ,), dtype='int32')
    embedding = Embedding(len(tokenizer.word_index)+1, EMBED_DIM,
                          weights=[embedding_matrix],
                          input_length = MAX_LEN_SEQ,
                          #mask_zero = True
                          trainable = False) (input_words)
    x = Dropout(DROPOUT_RATE)(embedding)
    x = Bidirectional(LSTM(128, activation='tanh', return_sequences=True))(x) #set return_sequence to false when passing to dense
    #x = Bidirectional(LSTM(128, activation='tanh', return_sequences=True))(x)
    
    # Use GlobalMaxPooling
    x = GlobalMaxPooling1D()(x)
    
    # Pass into DENSE layers 
    # Dense nodes total has been calculated as per 
    # https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm
    # (300,000)/5*(128+2) = 462
    x = Dense(462, activation='relu')(x)
    prediction = Dense(2, activation='sigmoid')(x)
    
    model = Model(inputs=input_words, outputs=prediction, name='baseline-LSTM')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC()])
    
    return model
                           
def train_model(train_df, val_df, tokenizer):
    # Create processed and padded train and targets
    print('padding_text')
    X_train = text_padder(train_df[TRAIN_TEXT_COL], tokenizer)
    X_val = text_padder(val_df[TRAIN_TEXT_COL], tokenizer)
    y_train = to_categorical(train_df[TRAIN_TARGET_COL])
    y_val = to_categorical(val_df[TRAIN_TARGET_COL])
    
    print('building embedding matrix')
    # build embedding matrix
    embed_matrix = build_embedding_matrix(tokenizer.word_index, EMBEDDING_FILE)
    
    # build model
    print('building model')
    model = build_model(embed_mat)
    
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH,
                                                 save_weights_only=True,
                                                 verbose=1)
    
    # Connect to tensorboard
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, write_image=True, write_graph=False
                                                          write_grad=True)
    # train model
    print('training model')
    fitted_model = model.fit(X_train, y_train,
                             batch_size = BATCH_SIZE,
                             epochs = NUM_EPOCHS,
                             validation_data=(X_val, y_val),
                             callbacks=[cp_callback, tensorboard_callback],
                             verbose = 1)
    #save full model 
    fitted_model.save('saved_model/baseline-LSTM') 
    #saves to h5
    fitted_model.save('saved_model/baseline-LSTM.h5')
    
    #save weights
    fitted_model.save_weights('saved_weights/baseline-LSTM')
    fitted_model.save_weights('saved_weights/baseline-LSTM.h5')
  
    return fitted_model
    
    
    

In [ ]:
def build_model_vocab_restricted(embedding_matrix):
    # change to max word length 
    input_words = Input(shape=(MAX_LEN_SEQ,), dtype='int32')
    embedding = Embedding(MAX_VOCAB_SIZE+1, EMBED_DIM,
                          weights=[embedding_matrix],
                          input_length = MAX_LEN_SEQ,
                          #mask_zero = True
                          trainable = False) (input_words)
    x = Dropout(DROPOUT_RATE)(embedding)
    x = Bidirectional(LSTM(128, activation='tanh', return_sequences=True))(x) #set return_sequence to false when passing to dense
    #x = Bidirectional(LSTM(128, activation='tanh', return_sequences=True))(x)
    
    # Use GlobalMaxPooling
    x = GlobalMaxPooling1D()(x)
    
    # Pass into DENSE layers 
    # Dense nodes total has been calculated as per 
    # https://ai.stackexchange.com/questions/3156/how-to-select-number-of-hidden-layers-and-number-of-memory-cells-in-an-lstm
    # (300,000)/5*(128+2) = 462
    x = Dense(462, activation='relu')(x)
    prediction = Dense(2, activation='sigmoid')(x)
    
    model = Model(inputs=input_words, outputs=prediction, name='baseline-LSTM')
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', AUC()])
    
    return model
                           
def train_model_restricted(train_df, val_df, tokenizer):
    # Create processed and padded train and targets
    print('padding_text')
    X_train = text_padder(train_df[TRAIN_TEXT_COL], tokenizer)
    X_val = text_padder(val_df[TRAIN_TEXT_COL], tokenizer)
    y_train = to_categorical(train_df[TRAIN_TARGET_COL])
    y_val = to_categorical(val_df[TRAIN_TARGET_COL])
    
    print('building embedding matrix')
    # build embedding matrix
    embed_matrix = build_embedding_matrix_restricted(tokenizer.word_index, EMBEDDING_FILE)
    
    # build model
    print('building model')
    model = build_model(embed_mat)
    
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH,
                                                 save_weights_only=True,
                                                 verbose=1)
    
    # Connect to tensorboard
    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, write_image=True, write_graph=False
                                                          write_grad=True)
    # train model
    print('training model')
    fitted_model = model.fit(X_train, y_train,
                             batch_size = BATCH_SIZE,
                             epochs = NUM_EPOCHS,
                             validation_data=(X_val, y_val),
                             callbacks=[cp_callback, tensorboard_callback],
                             verbose = 1)
    
    #save full model 
    fitted_model.save('saved_model/baseline-LSTM') 
    #saves to h5
    fitted_model.save('saved_model/baseline-LSTM.h5')
    
    #save weights
    fitted_model.save_weights('saved_weights/baseline-LSTM')
    fitted_model.save_weights('saved_weights/baseline-LSTM.h5')
    
    return fitted_model
    
    
    